In [60]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import os
import json
import datetime
from random import choice

In [72]:
desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']

def random_headers():
    return {'User-Agent': choice(desktop_agents)}

In [73]:
def get_buy_webpage(city, 
                    state, 
                    pg_num, 
                    htype=[
                        'house',
                        'multi-family',
                    ]):
    
    overhead = 'https://www.trulia.com'
    dangle = 'for_sale'

    city = city.title()\
               .replace(' ', '_')
    state = state.upper()

    dict_alias = {
        'house': 'SINGLE-FAMILY_HOME',
        'condo': 'APARTMENT,CONDO,COOP',
        'townhouse': 'TOWNHOUSE',
        'multi-family': 'MULTI-FAMILY',
        'land': 'LOT%7CLAND',
        'mobile/manufactured': 'MOBILE%7CMANUFACTURED',
        'other': 'UNKNOWN',
    }

    aliases = [dict_alias[h] for h in htype]
    houses = ','.join(aliases)

    webpage = f'{overhead}/{dangle}/{city},{state}/{houses}_type/{pg_num}_p/'
    return webpage


In [74]:
url = get_buy_webpage('philadelphia', 'pa', 1)
url

'https://www.trulia.com/for_sale/Philadelphia,PA/SINGLE-FAMILY_HOME,MULTI-FAMILY_type/1_p/'

In [209]:
def get_buy_apt_urls_per_page(city,
                              state,
                              pg_num,
                              htype=['house', 
                                     'multi-family']):

    webpage = get_buy_webpage(city, state, pg_num, htype)
    
    # Here we added User-Agent to the header of our request 
    # It is because sometimes the web server will check the
    # different fields of the header to block robot scrapers
    # User-Agent is the most common one because it is specific 
    # to your browser.
    soup = get_soup(webpage)
            
    apt_class = 'PropertyCard__PropertyCardContainer-sc-1ush98q-0 gsDQZj Box-sc-8ox7qa-0 jIGxjA'
    apt_tags = soup.find_all('div', class_=apt_class)
    
    print(apt_tags)

    apt_link_tags = [tag.find('a') for tag in apt_tags]
    apt_urls = [tag['href'] for tag in apt_link_tags]
    
    return apt_urls

def get_buy_apt_urls_ensemble(city,
                              state,
                              verbose=False,
                              htype=['house', 
                                     'multi-family']):
    stop = False
    urls_ensemble = ['']
    pg_num = 2
    while not stop:
        urls_per_pg = get_buy_apt_urls_per_page(city, state, pg_num, htype)
        
        if verbose:
            print(f'apartment URLs in page {pg_num} all done')
        
        if urls_per_pg == urls_ensemble[-1]:
            stop = True
        urls_ensemble.append(urls_per_pg)
        pg_num += 1
    
    def flatten(lst):
        for el in lst:
            if isinstance(el, list):
                yield from el
            else:
                yield el
    
    urls_ensemble_cut = list(flatten(urls_ensemble[1:-2]))
    if verbose:
        print('last 2 pages removed since they are the same.')
    return urls_ensemble_cut

In [210]:
get_buy_apt_urls_ensemble('philadelphia', 'pa')

[]
[]


[]

In [214]:
get_soup('https://www.trulia.com/for_sale/Philadelphia,PA/MULTI-FAMILY,SINGLE-FAMILY_HOME_type/')\
.find_all('div', class_='PropertyCard__PropertyCardContainer-sc-1ush98q-0 gsDQZj Box-sc-8ox7qa-0 jIGxjA')

[]

In [179]:
def get_buy_apt_urls(webpage):
    
    # Here we added User-Agent to the header of our request 
    # It is because sometimes the web server will check the
    # different fields of the header to block robot scrapers
    # User-Agent is the most common one because it is specific 
    # to your browser.
    soup = get_soup(webpage)
            
    apt_class = 'PropertyCard__PropertyCardContainer-sc-1ush98q-0 gsDQZj Box-sc-8ox7qa-0 jIGxjA'
    apt_tags = soup.find_all('div', class_=apt_class)
    
    apt_link_tags = [tag.find('a') for tag in apt_tags]
    apt_urls = [tag['href'] for tag in apt_link_tags]
    
    return apt_urls

In [224]:
get_buy_apt_urls('https://www.trulia.com/sold/Philadelphia,PA/')

[]

In [175]:
get_soup('https://www.trulia.com/sold/Philadelphia,PA/3_p/')

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><script>
          window.trulia = window.trulia || {};
          trulia.analytics = trulia.analytics || {};
          trulia.analytics.automaticTrackState = false;
        </script><meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0, viewport-fit=cover, minimal-ui" name="viewport"/><meta content="5953837487" property="fb:admins"/><meta content="183577541666001" property="fb:app_id"/><meta content="Trulia" property="application-name"/><meta content="https://www.trulia.com" property="msapplication-starturl"/><meta content="Trulia: Real Estate Search" property="msapplication-tooltip"/><meta content="/browserconfig.xml" property="msapplication-config"/><meta content="en_US" property="og:locale"/><meta content="Trulia Real Estate Search" property="og:site_name"/><meta content="website" property="og:type"/><meta content="//static.trulia-cdn.com/images/logos/standardized/v4/logo-with-icon-200x200.png" property="

In [228]:
get_soup('https://www.trulia.com/for_sale/Philadelphia,PA/MULTI-FAMILY,SINGLE-FAMILY_HOME_type/3_p')

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Access to this page has been denied.</title>
<style>
        @font-face {
            font-family: "PostGrotesk";
            src: url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.eot");
            src: url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.eot?#iefix") format("embedded-opentype"), url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.woff") format("woff"), url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.ttf") format("truetype"), url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.svg#PostGroteskRegular") format("svg");
            font-weight: 400;
            font-style: normal;
            font-display: swap;
        }

        @font-face {
            font-family: "P

In [174]:
get_buy_apt_urls_per_page('philadelpahi','pa',2)

[]


[]

In [93]:
import requests

url = "http://httpbin.org/ip"
proxy_host = "proxy.crawlera.com"
proxy_port = "8010"
proxy_auth = ":"
proxies = {
       "https": "https://{}@{}:{}/".format(proxy_auth, proxy_host, proxy_port),
       "http": "http://{}@{}:{}/".format(proxy_auth, proxy_host, proxy_port)
}

r = requests.get(url, proxies=proxies, verify=False)

<Response [407]>

In [15]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
apt_wb = 'https://www.trulia.com/p/pa/philadelphia/302-carpenter-ln-philadelphia-pa-19119--1037425672'
response = requests.get(apt_wb, headers=headers)
results = response.content
soup = BeautifulSoup(results, 'lxml')

In [111]:
def get_soup(url):
    
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'referrer': 'https://google.com',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Pragma': 'no-cache',
    }
    
    proxy_host = "proxy.crawlera.com"
    proxy_port = "8010"
    proxy_auth = ":"
    proxies = {
           "https": "https://{}@{}:{}/".format(proxy_auth, proxy_host, proxy_port),
           "http": "http://{}@{}:{}/".format(proxy_auth, proxy_host, proxy_port)
    }
    
    response = requests.get(url, headers={'user-agent':desktop_agents[2]})
    results = response.content
    if not response.status_code == 404:
        soup = BeautifulSoup(results, 'lxml')
    return soup

In [66]:
soup = get_soup(apt_wb)

In [67]:
jfile = soup.find('script', attrs={
    'id': '__NEXT_DATA__',
    'type': 'application/json',
}).get_text()


In [68]:
jdict = json.loads(jfile)
for key in jdict.keys():
    print(key)

dataManager
props
page
query
buildId
assetPrefix
runtimeConfig


In [20]:
jdict['props'].keys()

dict_keys(['abExperiments', 'abTracking', 'session', '_page', 'asPath', 'homeDetails', 'viewer', 'search', 'homeDetailsPromise', 'apolloState', 'apolloHeaders'])

In [21]:
jdict['props']['homeDetails'].keys()

dict_keys(['url', 'media', 'metadata', 'pageText', 'currentStatus', '__typename', 'tracking', 'primaryNavigation', 'secondaryNavigation', 'isSaveable', 'isShareable', 'preferences', 'location', 'adTargetings', 'price', 'heroTags', 'priceChange', 'bedrooms', 'bathrooms', 'floorSpace', 'provider', 'activeForRentListing', 'surroundings', 'hoaFee', 'mortgageInfo', 'nearbyHomes', 'features', 'publicRecord', 'comparables', 'description', 'titleToPriceHistory', 'priceHistory', 'localProtections', 'taxes', 'marketComparisons', 'breadcrumbNavigation', 'backToSearch', 'localInfoSummary', 'parcel', 'nearbyPointsOfInterest', 'seoDescription', 'similarHomes', 'foreclosureInfo', 'community', 'activeForSaleListing', 'assignedSchools', 'providerListingId', 'flagListingReportTypes', 'affordability', 'propertyType'])

In [22]:
jdict['props']['homeDetails']['media'].keys()

dict_keys(['metaTagHeroImages', '__typename', 'topThirdHeroImages', 'totalPhotoCount', 'threeDHomes', 'mapWithPin', 'videos', 'photos', 'streetView'])

In [23]:
jdict['props']['homeDetails']['media']['photos'][2]

{'url': {'extraSmallSrc': 'https://static.trulia-cdn.com/pictures/thumbs_5/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'smallSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'mediumSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'largeSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'hiDpiExtraSmallSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'hiDpiSmallSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'hiDpiMediumSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'hiDpiLargeSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  '__typename': 'MEDIA_ImageResource'},
 'webpUrl': {'extraSmallWebpSrc': 'https://static.trulia-cdn.com/

In [24]:
image_url = 'https://static.trulia-cdn.com/pictures/thumbs_5/zillowstatic/ISb91c89ycbre71000000000.jpg'
img_data = requests.get(image_url).content
with open('image_name.jpg', 'wb') as handler:
    handler.write(img_data)

In [25]:
jdict['props']

{'abExperiments': {'958': 'a',
  '1148': 'a',
  '1177': 'a',
  '1181': 'a',
  '1182': 'a',
  '1186': 'c'},
 'abTracking': '958a;1148a;1177a;1181a;1182a;1186c',
 'session': {'isBot': False,
  'userLifetimeId': '191120q197314jjw36imgjact82gm682',
  'requestId': '20ed2d8907826b0d8e8f785a9c5ae839',
  'navigatedFrom': False},
 '_page': {'title': '302 Carpenter Ln, Philadelphia, PA 19119 - 5 Bed, 3 Bath Single-Family Home - MLS# PAPH847616 - 29 Photos | Trulia',
  'description': "302 Carpenter Ln, Philadelphia, PA 19119 is a 3,012 sqft, 5 bed, 3 bath Single-Family Home listed for $650,000. Classic 3 story stone Queen Anne Victorian circa 1895 that has been updated for today's living! Features include a...",
  'metaTags': {'robots': {'name': 'robots', 'content': 'INDEX, FOLLOW'},
   'twitterCard': {'property': 'twitter:card', 'content': 'summary'},
   'twitterImageAlt': {'property': 'twitter:image:alt',
    'content': '302 Carpenter Ln, Philadelphia, PA 19119'},
   'ogUrl': {'property': 'og:u

In [26]:
def load_json(soup):
    jfile = soup.find('script', attrs={
        'id': '__NEXT_DATA__',
        'type': 'application/json',
    }).get_text()
    
    jdict = json.loads(jfile)
    return jdict

In [27]:
def get_img_urls(jdict):
    pics = jdict['props']['homeDetails']['media']['photos']
    urls = [pic['url']['mediumSrc'] for pic in pics]
    return urls

In [28]:
def save_images(img_urls, data_path, img_type, address):
    try:
        current_path = os.getcwd()
        os.chdir(data_path)
        
        if not os.path.exists(img_type):
            os.mkdir(img_type)
        os.chdir(img_type)
        
        if not os.path.exists(address):
            os.mkdir(address)
        os.chdir(address)
        
        for i, img_url in enumerate(img_urls):
            img_data = requests.get(img_url).content
            with open('img{i}.jpg', 'wb') as handler:
                handler.write(img_data)
                
        os.chdir(current_path)
        return 1
    
    except:
        return 0

In [32]:
test_jdict = load_json(soup)
test_img_urls = get_img_urls(test_jdict)
test_img_urls

['https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISvgw4ch9cdv430000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/IS7mojyi16a54a1000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISrhegouoc1k1l0000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISrxuf38nbz82l0000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISnqubpplomze71000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISfksz6jhge2f71000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/IS7eqnocd865f71000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISz7ob6690y7f71000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISr1mznz4spaf71000000000.jpg',
 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstat

In [33]:
save_images(test_img_urls, '../data/sample/trulia/imgdata', 'sold', 'test_address')

1

In [36]:
jdict['props']['homeDetails']['location']

{'stateCode': 'PA',
 '__typename': 'HOME_Location',
 'jsonLdSchemaFullLocation': '302 Carpenter Ln, Philadelphia, PA 19119',
 'homeFormattedAddress': '302 Carpenter Ln, Philadelphia, PA 19119',
 'cityStateZipAddress': 'Philadelphia, PA 19119',
 'summaryFormattedLocation': '302 Carpenter Ln',
 'city': 'Philadelphia',
 'zipCode': '19119',
 'neighborhoodName': 'Mount Airy West',
 'coordinates': {'latitude': 40.049873,
  'longitude': -75.191765,
  '__typename': 'HOME_Coordinates'},
 'streetAddress': '302 Carpenter Ln',
 'formattedLocation': '302 Carpenter Ln'}

In [9]:
def get_address(jdict):
    try:
        loc_dict = jdict['props']['homeDetails']['location']
        state = loc_dict['stateCode']
        city = loc_dict['city']
        zipcode = loc_dict['zipCode']
        street = loc_dict['formattedLocation']
        try:
            neighborhood = loc_dict['neighborhoodName']
        except:
            neighborhood = None
        return street, city, state, zipcode, neighborhood
    except:
        return None, None, None, None, None

In [38]:
get_address(jdict)

('302 Carpenter Ln', 'Philadelphia', 'PA', '19119', 'Mount Airy West')

In [40]:
jdict['props']['homeDetails']['price']

{'__typename': 'HOME_SinglePrice',
 'price': 650000,
 'currencyCode': 'USD',
 'formattedPrice': '$650,000',
 'branchBannerPrice': '$650K'}

In [32]:
def get_price(jdict):
    try:
        price_dict = jdict['props']['homeDetails']['price']
        return float(price_dict['price'])
    except:
        return None

def get_space(jdict): 
    try:
        space_dict = jdict['props']['homeDetails']['floorSpace']
        space_text = space_dict['formattedDimension'].replace(',','')
        space = extract_num(space_text)
        return space
    except:
        return np.nan

def get_bedrooms_bathrooms(jdict):
    try:
        pattern = r'[-+]?\d*\.\d+|\d+'
        bed_dict = jdict['props']['homeDetails']['bedrooms']
        bath_dict = jdict['props']['homeDetails']['bathrooms']
        bedrooms = re.findall(pattern, bed_dict['summaryBedrooms'])[0]
        bathrooms = re.findall(pattern, bath_dict['summaryBathrooms'])[0]
        return float(bedrooms), float(bathrooms)
    except:
        return np.nan, np.nan

def get_apt_features(jdict):
    try:
        fdict_list = jdict['props']['homeDetails']['features']['attributes']
        features = []
        for fdict in fdict_list:
            try:
                value = fdict['formattedValue']
                try:
                    key = fdict['formattedName']
                    features.append(f'{key}:{value}')
                except:
                    features.append(value)
            except:
                next
        return ' | '.join(features)
    except:
        return None

In [208]:
jdict['props']['homeDetails']['floorSpace']['formattedDimension']

'3,012 sqft'

In [215]:
get_space(jdict)

3012 sqft


3012.0

In [83]:
get_bedrooms_bathrooms(jdict)

(5.0, 3.0)

In [84]:
jdict['props']['homeDetails'].keys()

dict_keys(['url', 'media', 'metadata', 'pageText', 'currentStatus', '__typename', 'tracking', 'primaryNavigation', 'secondaryNavigation', 'isSaveable', 'isShareable', 'preferences', 'location', 'adTargetings', 'price', 'heroTags', 'priceChange', 'bedrooms', 'bathrooms', 'floorSpace', 'provider', 'activeForRentListing', 'surroundings', 'hoaFee', 'mortgageInfo', 'nearbyHomes', 'features', 'publicRecord', 'comparables', 'description', 'titleToPriceHistory', 'priceHistory', 'localProtections', 'taxes', 'marketComparisons', 'breadcrumbNavigation', 'backToSearch', 'localInfoSummary', 'parcel', 'nearbyPointsOfInterest', 'seoDescription', 'similarHomes', 'foreclosureInfo', 'community', 'activeForSaleListing', 'assignedSchools', 'providerListingId', 'flagListingReportTypes', 'affordability', 'propertyType'])

In [96]:
jdict['props']['homeDetails']['features']['attributes'][0]

{'formattedValue': 'Single Family Home',
 '__typename': 'HOME_FeatureAttributeGenericValueOnly'}

In [206]:
jdict['props']['homeDetails']['floorSpace']

{'formattedDimension': '3,012 sqft',
 '__typename': 'HOME_SingleDimension',
 'branchBannerFloorSpace': '3,012 sqft'}

In [95]:
get_apt_features(jdict)

'Single Family Home | $216/sqft | Lot Size:0.31 acres | Built in 1925 | 4 Days on Trulia | 10 Rooms | Rooms:Dining Room | Heating:Forced Air | Heating Fuel:Gas | Cooling System:Central | Air Conditioning | Washer | Dryer | Refrigerator | Floors:Hardwood | Great Views | Parking Spaces:3 | Basement | Victorian Architecture | Stories:3 | Exterior:Stone | MLS/Source ID:PAPH847616'

In [37]:
rent_soup = get_soup('https://www.trulia.com/c/pa/philadelphia/allegheny-apartments-1605-w-allegheny-ave-philadelphia-pa-19132--2089964389')

In [38]:
jdict_rent = load_json(rent_soup)
get_address(jdict_rent)

NameError: name 'get_address' is not defined

In [17]:
get_price(jdict_rent)

In [34]:
jdict_rent.keys()

dict_keys(['dataManager', 'props', 'page', 'query', 'buildId', 'assetPrefix', 'runtimeConfig'])

In [19]:
jdict_rent['props'].keys()

dict_keys(['abExperiments', 'abTracking', 'session', '_page', 'asPath', 'homeDetails', 'viewer', 'search', 'homeDetailsPromise', 'apolloState', 'apolloHeaders'])

In [41]:
jdict_rent['props']['homeDetails']['floorPlans']['floorPlanGroups'][0]['plans']

[{'name': 'Studio',
  'availabilityText': 'Ask for Availability',
  'leadFormMessagePlaceHolder': "I am interested in the Studio floor plan and other available Studios. I'd like to schedule a viewing. Please let me know when this would be possible.",
  'defaultPhoto': {'url': {'thumbnail': 'https://static.trulia-cdn.com/pictures/images/rentals/NoFloorplanThumb.png',
    'small': 'https://static.trulia-cdn.com/pictures/images/icons/no_fp_icon_320.jpg',
    'medium': None,
    '__typename': 'MEDIA_ImageResource'},
   '__typename': 'HOME_MediaPhoto'},
  'id': '48uzwcgqm6s9k',
  'bedrooms': {'formattedValue': 'Studio',
   'fullValue': 'Studio',
   '__typename': 'HOME_StudioBedroom'},
  'bathrooms': {'formattedValue': '1ba',
   'fullValue': '1 Bathroom',
   '__typename': 'HOME_FixedBathrooms'},
  'priceRange': {'formattedPrice': '$665', '__typename': 'HOME_PriceRange'},
  'floorSpace': {'formattedDimension': None,
   'max': None,
   '__typename': 'HOME_DimensionRange'},
  'contactCTAText': 

In [22]:
jdict_rent['props']['homeDetails']['price']

{'__typename': 'HOME_PriceRange',
 'min': 1930,
 'max': 4443,
 'currencyCode': 'USD',
 'formattedPrice': '$1,930 - $4,443/mo',
 'branchBannerPrice': '$1.93K+'}

In [24]:
jdict_rent['props']['homeDetails']['floorPlans'].keys()

dict_keys(['shortTitle', 'floorPlanGroups', '__typename'])

In [38]:
jdict_rent['props']['homeDetails']['floorPlans']['floorPlanGroups'][0]['plans'][2]['units']

[]

In [57]:
jdict_rent['props']['homeDetails']['floorPlans']['floorPlanGroups'][0]['plans'][0].keys()

dict_keys(['name', 'availabilityText', 'leadFormMessagePlaceHolder', 'defaultPhoto', 'id', 'bedrooms', 'bathrooms', 'priceRange', 'floorSpace', 'contactCTAText', 'priceUnavailableText', '__typename', 'units'])

In [76]:
jdict_rent['props']['homeDetails']['floorPlans']['floorPlanGroups'][0]['plans'][3]

{'name': 'Studio-S4',
 'availabilityText': 'Ask for Availability',
 'leadFormMessagePlaceHolder': "I am interested in the Studio-S4 floor plan and other available Studios. I'd like to schedule a viewing. Please let me know when this would be possible.",
 'defaultPhoto': {'url': {'thumbnail': 'https://static.trulia-cdn.com/pictures/thumbs_big/ps.114/8/6/3/6/picture-uh=9de2ca257471db32676e94db3d9e0-ps=8636e5729e567134b11f75d1064b40.jpg',
   'small': 'https://static.trulia-cdn.com/pictures/thumbs_3/ps.114/8/6/3/6/picture-uh=9de2ca257471db32676e94db3d9e0-ps=8636e5729e567134b11f75d1064b40.jpg',
   'medium': 'https://static.trulia-cdn.com/pictures/thumbs_4/ps.114/8/6/3/6/picture-uh=9de2ca257471db32676e94db3d9e0-ps=8636e5729e567134b11f75d1064b40.jpg',
   '__typename': 'MEDIA_ImageResource'},
  '__typename': 'HOME_MediaPhoto'},
 'id': '2197801',
 'bedrooms': {'formattedValue': 'Studio',
  'fullValue': 'Studio',
  '__typename': 'HOME_StudioBedroom'},
 'bathrooms': {'formattedValue': '1ba',
  'f

In [96]:
def get_section_data(section):
    apt_name = section['name']
            
    try:
        bedrooms_text = section['bedrooms']['fullValue']
        bathrooms_text = section['bathrooms']['fullValue']
        bedrooms = extract_num(bedrooms_text)
        bathrooms = extract_num(bathrooms_text)
    except:
        bedrooms, bathrooms = np.nan, np.nan

    try:
        space = float(section['floorSpace']['max'])
    except:
        space = np.nan

    try:
        price_text = section['priceRange']['formattedPrice']
        price_text = price_text.replace(',', '') \
                               .replace('$', '')
        price = extract_num(price_text)
    except:
        price = np.nan
        
    section_data = [
        apt_name,
        bedrooms,
        bathrooms,
        space,
        price,
    ]
    
    return section_data

def extract_num(text):
    try:
        if 'studio' in text.lower():
            return 0.0
        pattern = r'[-+]?\d*\.\d+|\d+'
        result = re.findall(pattern, text)[0]
        return float(result)
    except:
        return np.nan

def get_floorplans(jdict):
    floorplans_groups = jdict_rent['props']['homeDetails']['floorPlans']['floorPlanGroups']
    address_data = list(get_address(jdict))
    rental_data = []
    
    for floorplans in floorplans_groups:
        plans = floorplans['plans']
        for section in plans:
            section_data = get_section_data(section)
            rental_data.append(address_data+section_data)
            units = section['units']
            for unit in units:
                unit_data = get_section_data(unit)
                rental_data.append(address_data+unit_data)
    return rental_data

In [100]:
get_floorplans(jdict_rent)[-2]

['1213 Walnut St',
 'Philadelphia',
 'PA',
 '19107',
 'Washington Square',
 '2 Bed/2 Bath-B9',
 2.0,
 2.0,
 1155.0,
 nan]

In [93]:
jdict_rent['props']['homeDetails']['floorPlans']['floorPlanGroups'][0]['plans'][0]['units']

[{'name': 'Unit 1906',
  'availabilityText': 'Ask for Availability',
  'leadFormMessagePlaceHolder': "I am interested in Unit 1906, Studio Studio-S3 floor plan and other available Studios. I'd like to schedule a viewing. Please let me know when this would be possible.",
  'pricePeriodRange': {'formattedPrice': '$2,130/mo',
   '__typename': 'HOME_PriceRange'},
  'id': '1906',
  'bedrooms': {'formattedValue': 'Studio',
   'fullValue': 'Studio',
   '__typename': 'HOME_StudioBedroom'},
  'bathrooms': {'formattedValue': '1ba',
   'fullValue': '1 Bathroom',
   '__typename': 'HOME_FixedBathrooms'},
  'priceRange': {'formattedPrice': '$2,130', '__typename': 'HOME_PriceRange'},
  'floorSpace': {'formattedDimension': '433 sqft',
   'max': 433,
   '__typename': 'HOME_DimensionRange'},
  'contactCTAText': 'Contact',
  'priceUnavailableText': '',
  '__typename': 'HOME_RentalCommunityFloorplanUnit'},
 {'name': 'Unit 606',
  'availabilityText': 'Ask for Availability',
  'leadFormMessagePlaceHolder': 

In [227]:
soup_sold = get_soup('https://www.trulia.com/p/pa/philadelphia/6638-lotus-rd-philadelphia-pa-19151--1005541231')
soup_sold

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Access to this page has been denied.</title>
<style>
        @font-face {
            font-family: "PostGrotesk";
            src: url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.eot");
            src: url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.eot?#iefix") format("embedded-opentype"), url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.woff") format("woff"), url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.ttf") format("truetype"), url("//static.trulia-cdn.com/images/fonts/PostGroteskBook_20150623/PostGrotesk-Book.svg#PostGroteskRegular") format("svg");
            font-weight: 400;
            font-style: normal;
            font-display: swap;
        }

        @font-face {
            font-family: "P

In [226]:
jdict_sold = load_json(soup_sold)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [169]:
jdict_sold.keys()

dict_keys(['dataManager', 'props', 'page', 'query', 'buildId', 'assetPrefix', 'runtimeConfig'])

In [170]:
jdict_sold['props'].keys()

dict_keys(['abExperiments', 'abTracking', 'session', '_page', 'asPath', 'homeDetails', 'viewer', 'search', 'homeDetailsPromise', 'apolloState', 'apolloHeaders'])

In [171]:
jdict_sold['props']['homeDetails'].keys()

dict_keys(['url', 'media', 'metadata', 'pageText', 'currentStatus', '__typename', 'tracking', 'primaryNavigation', 'secondaryNavigation', 'isSaveable', 'isShareable', 'preferences', 'location', 'adTargetings', 'price', 'heroTags', 'priceChange', 'bedrooms', 'bathrooms', 'floorSpace', 'provider', 'activeForRentListing', 'surroundings', 'hoaFee', 'mortgageInfo', 'nearbyHomes', 'features', 'publicRecord', 'comparables', 'description', 'titleToPriceHistory', 'priceHistory', 'localProtections', 'taxes', 'marketComparisons', 'breadcrumbNavigation', 'backToSearch', 'localInfoSummary', 'parcel', 'nearbyPointsOfInterest', 'seoDescription', 'similarHomes', 'foreclosureInfo', 'community', 'activeForSaleListing', 'assignedSchools', 'providerListingId', 'flagListingReportTypes', 'affordability', 'propertyType'])

In [172]:
jdict_sold['props']['homeDetails']['price']

{'__typename': 'HOME_LastSoldPrice',
 'formattedPrice': '$35,000',
 'formattedSoldDate': 'Nov 15, 2019',
 'priceDifferencePercent': -0.02857142857142857,
 'formattedPriceDifferencePercent': '-3%',
 'listingPrice': {'formattedPrice': '$36K', '__typename': 'HOME_SinglePrice'},
 'pricePerDimension': {'formattedDimension': '$29/sqft',
  '__typename': 'HOME_SinglePriceDimension'},
 'branchBannerPrice': '$35K'}

In [219]:
def first(iterable, condition=lambda x: True):
    """
    Returns the first item in the 'iterable' that
    satisfies the 'condition'.

    If the condition is not given, returns the first item of
    the iterable.

    Raises 'StopIteration' if no item satysfing the condition is found.

    >>> first( (1,2,3), condition=lambda x: x % 2 == 0)
    2
    >>> first(range(3, 100))
    3
    >>> first( () )
    Traceback (most recent call last):
    ...
    StopIteration
    """
    try:
        return next(x for x in iterable if condition(x))
    except:
        return None

def get_historical_prices_dict(jdict):
    try:
        price_history = jdict['props']['homeDetails']['priceHistory']
        price_sold = first(price_history, 
                           condition=lambda x: x['event'].lower() == 'sold')
        price_listed = first(price_history,
                            condition=lambda x: x['event'].lower() == 'listed for sale')
        try:
            start, end = price_history.index(price_sold), price_history.index(price_listed)
            price_history = price_history[start:end]
            price_change = first(price_history,
                                condition=lambda x: x['event'].lower() == 'price change')
            return price_sold, price_change, price_listed
        except:
            return price_sold, None, price_listed
    except:
        return None, None, None

def unzip_pdict(price_dict):
    try:
        price_text = price_dict['price']['formattedPrice'].replace('$', '')\
                                                          .replace(',', '')
        price = extract_num(price_text)
        date = price_dict['formattedDate']
        date_formatted = datetime.datetime\
                                 .strptime(date, '%m/%d/%Y')\
                                 .strftime('%Y-%m-%d')
        return date_formatted, price
    except:
        return None, None

def get_normal_sold_prices(jdict):
    price_dict = jdict['props']['homeDetails']['price']
    try:
        sales_price_text = price_dict['formattedPrice'].replace(',','')\
                                                       .replace('$', '')
        sales_price = extract_num(sales_price_text)
        sales_date = price_dict['formattedSoldDate']
        sales_date_formatted = datetime.datetime\
                                       .strptime(sales_date, '%b %d, %Y')\
                                       .strftime('%Y-%m-%d')
        try:
            asking_price_text = price_dict['listingPrice']['formattedPrice'].replace(',','')\
                                                                            .replace('$', '')
            if 'k' in asking_price_text.lower():
                asking_price = extract_num(asking_price_text)*1e3
            elif 'm' in asking_price_text.lower():
                asking_price = extract_num(asking_price_text)*1e6
            else:
                asking_price = extract_num(asking_price_text)
            return sales_date_formatted, sales_price, asking_price
        except:
            return sales_date_formatted, sales_price, np.nan
    except:
        return None, None, None
    

def get_important_sold_prices(jdict):
   
    pdict_s, pdict_c, pdict_l = get_historical_prices_dict(jdict)
    date_s, price_s = unzip_pdict(pdict_s)
    date_c, price_c = unzip_pdict(pdict_c)
    date_l, price_l = unzip_pdict(pdict_l)
    
    return date_s, price_s, date_c, price_c, date_l, price_l

def get_sold_info(jdict):
    street, city, state, zipcode, neighborhood = get_address(jdict)
    bedrooms, bathrooms = get_bedrooms_bathrooms(jdict)
    space = get_space(jdict)
    features = get_apt_features(jdict)
    sales_date, sales_price, ask_price = get_normal_sold_prices(jdict)
    sold_date, sold_price, change_date, change_price, list_date, list_price = get_important_sold_prices(jdict)
    
    sold_info = [
        street, 
        city, 
        state, 
        zipcode, 
        neighborhood,
        bedrooms, 
        bathrooms,
        space,
        features,
        sales_date, 
        sales_price, 
        ask_price,
        sold_date, 
        sold_price, 
        change_date, 
        change_price, 
        list_date, 
        list_price,
    ]
    
    return sold_info

In [220]:
get_sold_info(jdict_sold)

['4419 Wayne Ave',
 'Philadelphia',
 'PA',
 '19140',
 'Logan',
 3.0,
 1.0,
 1188.0,
 'Townhouse | $29/sqft | Lot Size:1,035 sqft | Built in 1940 | 6 Rooms | Rooms:Dining Room | Heating:Forced Air | Heating Fuel:Gas | Cooling System:Central | Air Conditioning | Great Views | Colonial Architecture | Stories:2 | Exterior:Brick | Disabled Access',
 '2019-11-15',
 35000.0,
 36000.0,
 '2019-11-15',
 35000.0,
 '2019-10-26',
 36000.0,
 '2019-10-10',
 45000.0]

In [193]:
if not first([1,2,3,4,5], condition = lambda x: x>100):
    print('yes')

yes


In [194]:
get_important_prices(jdict_sold)

('2019-11-15', 35000.0, '2019-10-26', 36000.0, '2019-10-10', 45000.0)

In [195]:
get_normal_prices(jdict_sold)

('2019-11-15', 35000.0, 36000.0)

In [187]:
1e3

1000.0

In [198]:
get_apt_features(jdict_sold)

'Townhouse | $29/sqft | Lot Size:1,035 sqft | Built in 1940 | 6 Rooms | Rooms:Dining Room | Heating:Forced Air | Heating Fuel:Gas | Cooling System:Central | Air Conditioning | Great Views | Colonial Architecture | Stories:2 | Exterior:Brick | Disabled Access'

In [42]:
soup_sold = get_soup('https://www.trulia.com/sold/Philadelphia,PA/2_p/')

In [46]:
main_div = soup_sold.find_all('div', class_='PropertyCard__PropertyCardContainer-sc-1ush98q-0 gsDQZj Box-sc-8ox7qa-0 jIGxjA')

In [47]:
apt_link_tags = [tag.find('a') for tag in main_div]

In [51]:
[f"https://www.trulia.com{tag['href']}" for tag in apt_link_tags]

['https://www.trulia.com/p/pa/philadelphia/5701-leonard-st-philadelphia-pa-19149--2017393036',
 'https://www.trulia.com/p/pa/philadelphia/1839-e-albert-st-philadelphia-pa-19125--2089944659',
 'https://www.trulia.com/p/pa/philadelphia/58103-delaire-landing-rd-103-philadelphia-pa-19114--2017162627',
 'https://www.trulia.com/p/pa/philadelphia/3723-l-st-philadelphia-pa-19124--2017213222',
 'https://www.trulia.com/p/pa/philadelphia/6242-n-park-ave-philadelphia-pa-19141--1119549782',
 'https://www.trulia.com/p/pa/philadelphia/2451-cedar-st-philadelphia-pa-19125--2017229757',
 'https://www.trulia.com/p/pa/philadelphia/1615-s-21st-st-philadelphia-pa-19145--2090069068',
 'https://www.trulia.com/p/pa/philadelphia/930-e-mcpherson-st-philadelphia-pa-19150--2090107017',
 'https://www.trulia.com/p/pa/philadelphia/5025-beaumont-ave-philadelphia-pa-19143--2017332652',
 'https://www.trulia.com/p/pa/philadelphia/1101-locust-st-7d-philadelphia-pa-19107--2017134805',
 'https://www.trulia.com/p/pa/philadel